# logistic regression

# 1.using statsmodels (statistics)

In [ ]:
y=df['TenYearCHD']
x=df.drop('TenYearCHD',axis=1)

In [ ]:
import statsmodels.api as sm
X=sm.add_constant(x)
model=sm.Logit(y,X)
result=model.fit()
result.summary()

In [ ]:
##for checking multicollinearity##      vif method
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
pd.DataFrame({'vif': vif[0:]}, index=X.columns).T

In [ ]:
cols = list(X.columns)
p= []
while (len(cols)>1):
    
    X = X[cols]
                                ##if constant is added already, remove this.
    model = sm.Logit(y,X).fit().pvalues
    p = pd.Series(model.values[1:],index =X.columns[1:])      
    pmax = max(p)
    pid=p.idxmax()
    if(pmax>0.05):
        cols.remove(pid)
        print('variable removed:',pid,pmax)
    else:
        break
cols


##already the remianing columns are removed from the dataframe. run sm.logit directly

In [ ]:
model=sm.Logit(y,X)
result=model.fit()
result.summary()

In [ ]:
# for parameters or coeffecients

In [ ]:
result.params

In [ ]:
result.predict() ## for predicting the test data set.

In [ ]:
#for taking exponential value for assumption number three:
np.exp(result.params)

In [ ]:
RevolvingUtilizationOfUnsecuredLines    4.204060
age                                     0.945293
DebtRatio                               0.725279
MonthlyIncome                           0.999896
NumberOfOpenCreditLinesAndLoans         1.003080
NumberRealEstateLoansOrLines            1.167603
NumberOfDependents                      1.032890

#### assumptions from logistic regression model (coeffecient summary)

inference from model:  (in linear regression connection is straigh forward unlike logistic regression)
#checking signs
1) when age increases chance of delinquency decreases
2) when debtratio increases chance of delinquency increases.

#checking log(odds)
1) as age increases by 1 year,log(odds(deliquency)) decreases by .02
2) as number of dependents increases by 1,log(odds(deliquency)) increases by .08.

#checking exponential 
1) as age increases by 1 year, odds(delinquency)decreases by 0.02 (2%) (1-.02)
2) as number of dependents increases by 1, odds(delinquency) increases by 0.08 (8%)
3)in case of RevolvingUtilizationOfUnsecuredLines,DebtRatio etc, it shows more than 1. in this cases we need to modify the features before modelling so tnat we get meaniful ratios here.

# confusion matrix for statsmodel

In [ ]:
prob=result.predict(X[cols])
prob=pd.DataFrame(prob,columns=['prob'])
prob['y_est']=prob['prob'].apply(lambda x:1 if x>=0.5 else 0)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
confusion_matrix(y,prob['y_est'])

In [ ]:
accuracy_score(y,prob['y_est'])

In [ ]:
print(classification_report(y,prob['y_est']))

# 2.using sklearn(machine learning)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x, y, test_size=0.3, random_state=42)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression(solver='liblinear',fit_intercept=True)

In [ ]:
logreg.fit(x_train,y_train)

In [ ]:
#for coeffecients
logreg.coef_

In [ ]:
#to find accuracy from confusion matrix: we need to filter our model's probability by a threshold and compare with actual y_test value.

y_prob=logreg.predict_proba(x_test)  # gives model's prediction. take logreg.predict_proba(x_test)[:,1], for probability of y being 1
y_pred=logreg.predict(x_test)        # filters the y_prob by threshold 0.5 defualt. below code for changing 0.5 to other values.


from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
confusion_matrix(y_test,y_pred)
accuracy_score(y_test,y_pred)
print(classification_report(y,pro
                            b['y_est']))

print('\n','confusion matrix - Test : ','\n',confusion_matrix(y_test,y_pred) )
print('overall accuracy - Test : ',accuracy_score(y_test,y_pred) )

In [ ]:
#when you need to manually put threshold and calculate y_pred:
yprob=pd.DataFrame(y_prob[:,1],columns=['yprob'])
yprob['ypred']=yprob['yprob'].apply(lambda x: 1 if x>0.4 else 0)   ##threshold 0.4 here.
confusion_matrix(y_test,yprob['ypred'])
accuracy_score(y_test,yprob['ypred'])


In [ ]:
##to plot the curve to understand the concept of threshold(0.5),tpr,fpr :

y_prob_test=logreg.predict_proba(x_test)[:,1]   #slicing done to select only probability of (target being 1)
prob_plot=pd.DataFrame([y_prob_test,y_test]).T

prob_plot1=prob_plot[(prob_plot[1]==0)]  # filtering to get probability of target when actual value is 0 and 1
prob_plot2=prob_plot[(prob_plot[1]==1)]

sns.distplot(prob_plot1[0])
sns.distplot(prob_plot2[0])
plt.show()

In [ ]:
#ROC curve:
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression(solver='liblinear',fit_intercept=True)
logreg.fit(x_train,y_train)
y_prob_train=logreg.predict_proba(x_train)[:,1]
y_pred_train=logreg.predict(x_train)
y_prob=logreg.predict_proba(x_test)[:,1]
y_pred=logreg.predict(x_test)

from sklearn.metrics import roc_auc_score,roc_curve
roc_auc_score(y_train,y_prob_train)


from sklearn.metrics import roc_auc_score,roc_curve

print('AUC of train  : ',roc_auc_score(y_train,y_prob_train))
fpr,tpr,thresholds=roc_curve(y_train,y_prob_train)
plt.plot(fpr,tpr)
plt.plot(fpr,fpr,'r-') ##for the diagnal line
plt.xlabel('fpr')
plt.ylabel('tpr')
plt.title('ROC of train data')
plt.show()


In [ ]:
##ROC curve- same as above. micro level details added here.

from sklearn.metrics import roc_auc_score,roc_curve

print('AUC of train  : ',roc_auc_score(y_test,y_test_proba))
                                    ##origina value vs our prediction value
                                    #make sure y_test_proba=logreg.predict_proba(x_test)[:,1]. slicing for taking probability of y=1

fpr,tpr,thresholds=roc_curve(y_test,y_test_proba)  #make sure y_test/y_train is not dataframe. it should be just array. 
                                                    #just drop y feature in the dataset.
plt.plot(fpr,tpr)
plt.plot(fpr,fpr,'r-') ##for the diagnal line
plt.xlabel('fpr')
plt.ylabel('tpr')
plt.title('ROC of train data')
plt.show()


# ------------------------------------------------------------------------------------------------

# decission tree

In [ ]:
from sklearn.model_selection import train_test_split
X = df.drop('TenYearCHD', axis=1)
y = df['TenYearCHD']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(max_depth=3, min_samples_leaf=10 )
dt.fit(X, y)

In [ ]:
from IPython.display import Image
from sklearn.externals.six import StringIO
from sklearn.tree import export_graphviz
import pydotplus


features = X.columns
dot_data = export_graphviz(dt, out_file=None, feature_names=features)
graph = pydotplus.graph_from_dot_data(dot_data)
Image(graph.create_png())

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

y_pred_train = dt.predict(X_train)   ##takes 0.5 as threshold default. us manual program if threshold to be changed from 0.5
y_pred_test = dt.predict(X_test)
y_prob_test = dt.predict_proba(X_test)[:,1]
y_prob_train = dt.predict_proba(X_train)[:,1]

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

print('Accuracy of Decision Tree-Train: ', accuracy_score(y_pred_train, y_train))
print('Accuracy of Decision Tree-Test: ', accuracy_score(y_pred_test, y_test))

In [ ]:
auc-roc:
print('AUC of train  : ',roc_auc_score(y_train,y_prob_train[:,1]))

fpr,tpr,thresholds=roc_curve(y_train,y_prob_train[:,1])
plt.plot(fpr,tpr)
plt.plot(fpr,fpr,'r-') ##for the diagnal line
plt.xlabel('fpr')
plt.ylabel('tpr')
plt.title('ROC of train data')
plt.show()

In [ ]:
#gini of each leaf:
gini_1stleaf=1-(probability of yes)**2-(probability of no)**2
gini_2ndleaf=1-(probability of yes)**2-(probability of no)**2

gini of the column having above two leaf=weighted average of above two.
{1stleaf('0's+'1's)/both_leaf's('0's+'1's)}*gini_1stleaf        +      {2ndleaf('0's+'1's)/both_leaf's('0's+'1's)}*gini_2ndleaf    
 



# hyperparameter tuning in Decission Tree

# 1.grid search

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [ ]:
dt = DecisionTreeClassifier()

params = {'max_depth' : [2,3,4,5,6,7,8],
        'min_samples_split': [2,3,4,5,6,7,8,9,10],
        'min_samples_leaf': [1,2,3,4,5,6,7,8,9,10]}

gsearch = GridSearchCV(dt, param_grid=params, cv=2)

gsearch.fit(X,y)

gsearch.best_params_

In [ ]:
use these parameter to build the decission tree

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)


dt = DecisionTreeClassifier(**gsearch.best_params_)
dt.fit(X_train, y_train)

y_pred_train = dt.predict(X_train)
y_prob_train = dt.predict_proba(X_train)[:,1]

y_pred = dt.predict(X_test)
y_prob = dt.predict_proba(X_test)[:,1]

from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

print('Accuracy of Decision Tree-Train: ', accuracy_score(y_pred_train, y_train))
print('Accuracy of Decision Tree-Test: ', accuracy_score(y_pred, y_test))
print('AUC of Decision Tree-Train: ', roc_auc_score(y_train, y_prob_train))
print('AUC of Decision Tree-Test: ', roc_auc_score(y_test, y_prob))

# 2.Random Search

In [ ]:
from scipy.stats import randint as sp_randint
dt = DecisionTreeClassifier(random_state=1)
params = {'max_depth' : sp_randint(2,10),
        'min_samples_split': sp_randint(2,50),
        'min_samples_leaf': sp_randint(1,20),
         'criterion':['gini', 'entropy']}
rand_search = RandomizedSearchCV(dt, param_distributions=params, cv=3, random_state=1)
rand_search.fit(X, y)
print(rand_search.best_params_)

build a decision tree with these parameters.

In [ ]:
dt = DecisionTreeClassifier(**rand_search.best_params_)
dt.fit(X_train, y_train)

y_pred_train = dt.predict(X_train)
y_prob_train = dt.predict_proba(X_train)[:,1]
y_pred = dt.predict(X_test)
y_prob = dt.predict_proba(X_test)[:,1]

from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

print('Accuracy of Decision Tree-Train: ', accuracy_score(y_pred_train, y_train))
print('Accuracy of Decision Tree-Test: ', accuracy_score(y_pred, y_test))
print('AUC of Decision Tree-Train: ', roc_auc_score(y_train, y_prob_train))
print('AUC of Decision Tree-Test: ', roc_auc_score(y_test, y_prob))

# --------------------------------------------------------------------------------------------------------

# Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(random_state=1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(random_state=1)

rfc.fit(X_train,y_train)


y_pred_train=rfc.predict(X_train)
y_prob_train=rfc.predict_proba(X_train)[:,1]
y_pred_test=rfc.predict(X_test)
y_prob_test=rfc.predict_proba(X_test)[:,1]

from sklearn.metrics import accuracy_score,roc_auc_score,roc_curve
print('accuracy of random forest-train  :  ', accuracy_score(y_pred_train,y_train))
print('accuracy of random forest-test  :  ', accuracy_score(y_pred_test,y_test))
print('AUC of random forest-train  :  ', roc_auc_score(y_train,y_prob_train))
print('AUC of random forest-test  :  ', roc_auc_score(y_test,y_prob_test))

# hyperparameter tuning for RF

In [ ]:
from scipy.stats import randint as sp_randint
rfc=RandomForestClassifier(random_state=1)

params={'n_estimators':sp_randint(5,25),
    'criterion':['gini','entropy'],
    'max_depth':sp_randint(2,10),
    'min_samples_split':sp_randint(2,20),
    'min_samples_leaf':sp_randint(1,20),
    'max_features':sp_randint(2,15)}

rand_search_rfc=RandomizedSearchCV(rfc,param_distributions=params,
                               cv=3,random_state=1)


rand_search_rfc.fit(X,y)
print(rand_search_rfc.best_params_)

In [ ]:
now make random forest using above optimal parameters

In [ ]:
#from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(**rand_search_rfc.best_params_)

rfc.fit(X_train,y_train)


y_pred_train=rfc.predict(X_train)
y_prob_train=rfc.predict_proba(X_train)[:,1]
y_pred_test=rfc.predict(X_test)
y_prob_test=rfc.predict_proba(X_test)[:,1]

from sklearn.metrics import accuracy_score,roc_auc_score,roc_curve
print('accuracy of random forest-train  :  ', accuracy_score(y_pred_train,y_train))
print('accuracy of random forest-test  :  ', accuracy_score(y_pred_test,y_test))
print('AUC of random forest-train  :  ', roc_auc_score(y_train,y_prob_train))
print('AUC of random forest-test  :  ', roc_auc_score(y_test,y_prob_test))

In [ ]:
fpr,tpr,thresholds=roc_curve(y_test,y_prob_train)

plt.plot(fpr,tpr)
plt.plot(fpr,fpr,'r-')
plt.xlabel('Fpr')
plt.ylabel('Tpr')
plt.show()

# ---------------------------------------------------------------------------------------------------

# k-NN classifier
always standardize the data

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

In [ ]:
#  here transformation of all featuers (only) are done. not target.
#  transformation is based on zscore.

from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

Xs = ss.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

X_trains = ss.fit_transform(X_train)
X_tests = ss.transform(X_test)

In [ ]:
knn.fit(X_trains, y_train)

y_pred_train = knn.predict(X_trains)
y_prob_train = knn.predict_proba(X_trains)[:,1]

y_pred_test = knn.predict(X_tests)
y_prob_test = knn.predict_proba(X_tests)[:,1]

from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

print('Accuracy of kNN-Train: ', accuracy_score(y_pred_train, y_train))
print('Accuracy of kNN-Test: ', accuracy_score(y_pred_test, y_test))
print('AUC of kNN-Train: ', roc_auc_score(y_train, y_prob_train))
print('AUC of kNN-Test: ', roc_auc_score(y_test, y_prob_test))

### hyperparameter tuning for k-NN

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
knn = KNeighborsClassifier()

params = {'n_neighbors': sp_randint(1,20),
        'p': sp_randint(1,5)}

rand_search_knn = RandomizedSearchCV(knn, param_distributions=params,
                                 cv=3, random_state=1)
rand_search_knn.fit(Xs, y)
print(rand_search_knn.best_params_)

now make a new k-NN model with all above optimum parameters

In [ ]:
knn = KNeighborsClassifier(**rand_search_knn.best_params_)

knn.fit(X_trains, y_train)

y_pred_train = knn.predict(X_trains)
y_prob_train = knn.predict_proba(X_trains)[:,1]

y_pred_test = knn.predict(X_tests)
y_prob_test = knn.predict_proba(X_tests)[:,1]

from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

print('Accuracy of kNN-Train: ', accuracy_score(y_pred_train, y_train))
print('Accuracy of kNN-Test: ', accuracy_score(y_pred_test, y_test))
print('AUC of kNN-Train: ', roc_auc_score(y_train, y_prob_train))
print('AUC of kNN-Test: ', roc_auc_score(y_test, y_prob_test))

### finding optimum number of neighbour using MSE(miss classification error)

In [ ]:
# creating odd list of K for KNN
myList = list(range(1,20))

# subsetting just the odd ones
neighbors = list(filter(lambda x: x % 2 != 0, myList))


# empty list that will hold accuracy scores
ac_scores = []

# perform accuracy metrics for values from 1,3,5....19
for k in neighbors:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    # predict the response
    y_pred = knn.predict(X_test)
    # evaluate accuracy
    scores = accuracy_score(y_test, y_pred)
    ac_scores.append(scores)

# changing to misclassification error
MSE = [1 - x for x in ac_scores]

# determining best k
optimal_k = neighbors[MSE.index(min(MSE))]
print("The optimal number of neighbors is %d" % optimal_k)

#### plotting missclassification error vs number_neighbours

In [ ]:
## miss classification eror(MSE)  is 1-accuracy score. find it for each iteartion and plot graph.

import matplotlib.pyplot as plt
# plot misclassification error vs k
plt.plot(neighbors, MSE)
plt.xlabel('Number of Neighbors K')
plt.ylabel('Misclassification Error')
plt.show()

# ---------------------------------------------------------------------------------------------

# Stacking Algorithms

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
lr=LogisticRegression(solver='liblinear')
rfc=RandomForestClassifier(**rand_search_rfc.best_params_)
knn=KNeighborsClassifier(**rand_search_knn.best_params_)


In [ ]:
clf=VotingClassifier(estimators=[('lr',lr),('rfc',rfc),('knn',knn)])
clf.fit(X_train,y_train)

In [ ]:
clf=VotingClassifier(estimators=[('lr',lr),('rfc',rfc),('knn',knn)],voting='soft',weights=[2,3,1])  #for lr weigtage is 2/5-40%. for rf, 3/6=33%, for knn 1/6%
                                                                                                    #'remove weights=' for equal weightage.
clf.fit(X_train,y_train)

y_pred_train=clf.predict(X_train)
y_prob_train=clf.predict_proba(X_train)[:,1]

#y_pred=clf.predict(X_test)
y_prob=clf.predict_proba(X_test)[:,1]

In [ ]:
print('accuracy of stacked algos-train  :  ', accuracy_score(y_pred_train,y_train))
print('accuracy of stacked algos-test  :  ', accuracy_score(y_pred,y_test))
print('AUC of stacked algos-train  :  ', roc_auc_score(y_train,y_prob_train))
print('AUC of stacked algos-test  :  ', roc_auc_score(y_test,y_prob))

# ---------------------------------------------------------------------------------------------------

# gradient boosting machines

# light gradient boosting machine:

In [ ]:
import lightgbm as lgb
lgbc=lgb.LGBMClassifier()
X=
y=
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.3, random_state=1)


In [ ]:
lgbc.fit(X_train,y_train)
y_pred=lgbc.predict(X_test)
y_prob=lgbc.predict_proba(X_test)[:,1]

In [ ]:
from sklearn.metrics import accuracy_score,roc_auc_score,roc_curve
print('Accuracy on test set  ', accuracy_score(y_test,y_pred))
print('AUC on test set  ', roc_auc_score(y_test,y_prob))

In [ ]:
cols=X.columns
lgbc.feature_importances_
fi=pd.DataFrame(lgbc.feature_importances_,index=cols,columns=['importance'])
fi

In [ ]:
cols=X.columns
lgbc.feature_importances_
fi=pd.DataFrame(lgbc.feature_importances_,index=cols,columns=['importance'])
fi=fi.sort_values('importance',ascending=False)
fi.plot.bar()
plt.show()

# hyperparamter tuning for lgbm

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
lgbc=lgb.LGBMClassifier(random_state=1)




params={'n_estimators':sp_randint(5,250),
        'max_depth':sp_randint(2,20),
        'min_samples_split':sp_randint(2,20),
        'min_samples_leaf':sp_randint(1,20),
        'num_leaves':sp_randint(5,50)}

rand_search_lgbc=RandomizedSearchCV(lgbc,param_distributions=params,
                               cv=3,random_state=1)


rand_search_lgbc.fit(X,y)
print(rand_search_lgbc.best_params_)

In [ ]:
lgbc=lgb.LGBMClassifier(**rand_search_lgbc.best_params_,random_state=1)
lgbc.fit(X_train,y_train)
    
y_pred_train=lgbc.predict(X_train)
y_prob_train=lgbc.predict_proba(X_train)[:,1]
    
y_pred=lgbc.predict(X_test)
y_prob=lgbc.predict_proba(X_test)[:,1]
    
from sklearn.metrics import accuracy_score,roc_auc_score,roc_curve
    
print('Accuracy of LGBM-Train ',accuracy_score(y_pred_train,y_train) )
print('Accuracy of LGBM-test',accuracy_score(y_pred,y_test) )
    
    
print('AUC ofLGBM-train ',roc_auc_score(y_train,y_prob_train) )
print('AUC of LGBM-Test ',roc_auc_score(y_test,y_prob) )
    
    

In [ ]:
fpr,tpr,thresholds=roc_curve(y_test,y_prob)
plt.plot(fpr,tpr)
plt.plot(fpr,fpr,'r-')
plt.show()

# --------------------------------------------------------------------------------------------

# Support Vector Machines (SVM)

In [ ]:
from sklearn.svm import SVC
svc=SVC(gamma='auto',probability=True)

svc.fit(X_train,y_train)


y_pred_train=svc.predict(X_train)
y_prob_train=svc.predict_proba(X_train)[:,1]
y_pred=svc.predict(X_test)
y_prob=svc.predict_proba(X_test)[:,1]

from sklearn.metrics import accuracy_score,roc_auc_score,roc_curve
print('accuracy of svc-train  :  ', accuracy_score(y_pred_train,y_train))
print('accuracy of svc-test  :  ', accuracy_score(y_pred,y_test))
print('auc of svc-train  :  ', roc_auc_score(y_train,y_prob_train))
print('auc of svc-test  :  ', roc_auc_score(y_test,y_prob))

# hyperparameter tuning for svm

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
svc=SVC(probability=True)

params={'C': np.logspace(-4,4,10000),
       'gamma':np.logspace(-4,4,10000)}

rand_search_svc=RandomizedSearchCV(svc,param_distributions=params,
                               cv=3,random_state=1)

rand_search_svc.fit(X,y)
print(rand_search_svc.best_params_)

In [ ]:
from sklearn.svm import SVC
svc=SVC(**rand_search_svc.best_params_,probability=True,random_state=1)
svc.fit(X_train,y_train)
    
y_pred_train=svc.predict(X_train)
y_prob_train=svc.predict_proba(X_train)[:,1]
    
y_pred=svc.predict(X_test)
y_prob=svc.predict_proba(X_test)[:,1]
    
from sklearn.metrics import accuracy_score,roc_auc_score,roc_curve
    
print('Accuracy of svc-Train ',accuracy_score(y_pred_train,y_train) )
print('Accuracy of svc-test',accuracy_score(y_pred,y_test) )
    
    
print('AUC of svc-train ',roc_auc_score(y_train,y_prob_train) )
print('AUC of svc-Test ',roc_auc_score(y_test,y_prob) )
    
    

# ---------------------------------------------------------------------------------------

# naive bayes classifier

In [ ]:
df['Class']=df['Class'].map({'ham':0,'spam':1})   ##making two classes of target into 0 and 1

In [ ]:
from sklearn.model_selection import train_test_split

X=df['sms']
y=df['Class']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=1)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vect=CountVectorizer(stop_words='english')   ##stop_words=englsh, will remove stop words such asconnecting words like if,else,so etc

In [ ]:
vect.fit(X_train)
X_train_tr=vect.transform(X_train)
X_test_tr=vect.transform(X_test)


In [ ]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score,roc_auc_score
bnb=BernoulliNB()

bnb.fit(X_train_tr,y_train)
y_pred=bnb.predict(X_test_tr)
y_pred_proba=bnb.predict_proba(X_test_tr)[:,1]

print('Accuracy score on Test  : ', accuracy_score(y_test,y_pred))
print('AUC on Test  : ', roc_auc_score(y_test,y_pred_proba))

# Analysis of different algorithms

In [ ]:
df_analysis=pd.DataFrame({'logistic regression':[lr_accuracy,lr_roc],'random forrest':[rf_accuracy,rf_roc],'K-NN':[knn_accuracy,knn_roc]},index=['accuracy_value','roc-auc_value'])

df_analysis
df_analysis.plot(kind='bar')
plt.show()

# Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['status']=le.fit_transform(df['status'])

# Handling Imbalance Data(undersampling,oversampling,SMOTE)

In [ ]:
say we got [0]-.998% and [1]-.001%, we have imbalance Data here.
#we will do treatment of imbalance data in train data only. let model learn from it. and apply this model on test data as it is.

##### 1.Undersampling Majority Class

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
Xy_train=pd.concat([X_train,y_train],axis=1)
#splitting 0,1 now
Xy_train0=Xy_train.loc[Xy_train['Class']==0]
Xy_train1=Xy_train.loc[Xy_train['Class']==1]
#we will undersample 0 now

len1=len(Xy_train1)
len0=len(Xy_train0)

Xy_train0_us=Xy_train0.sample(len1)
Xy_train_us=pd.concat([Xy_train0_us,Xy_train1],axis=0)

y_train_us=Xy_train_us['Class']
X_train_us=Xy_train_us.drop('Class',axis=1)

In [ ]:
len(y_train_us)  ##we made zeroes to match with 1. hence 357+357=714
print(len(y_train),len(y_test))
print(len(y_train[y_train==1]),len(y_test[y_test==1]))

we undersampled the majority class (0) to make it equal to minority class(1).
this exercise was done only for train set.test set was left untouched.
build the model in rtain set and try in test set. check whether there is improvement from the results of model of imbalanced data.

In [ ]:
rfc=RandomForestClassifier(**rand_search_rfc.best_params_)

model_compare(rfc,X_train_us,X_test,y_train_us,y_test)

##### 2.Oversampling of Minority Class

In [ ]:
Xy_train=pd.concat([X_train,y_train],axis=1)  ##x and y data together
#splitting two datasets of 0,1 now
Xy_train0=Xy_train.loc[Xy_train['Class']==0]
Xy_train1=Xy_train.loc[Xy_train['Class']==1]

#we will oversample 1 to make it as same as 0.
len1=len(Xy_train1)
len0=len(Xy_train0)


Xy_train1_os=Xy_train1.sample(len0,replace=True)
Xy_train_os=pd.concat([Xy_train1_os,Xy_train0],axis=0) #bringing togther noramal data and oversample data together

y_train_os=Xy_train_os['Class']
X_train_os=Xy_train_os.drop('Class',axis=1)

In [ ]:
print(len(y_train_os))   ##we made 1 to match with zeroes. so 2 lakh became 4 lakhs here.there were almost 2 lakh zeroes and 357 '1'

In [ ]:
rfc=RandomForestClassifier(**rand_search_rfc.best_params_)

model_compare(rfc,X_train_os,X_test,y_train_os,y_test)

##### handling imbalance data-SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE
smote=SMOTE(ratio='minority')
X_train_sm,y_train_sm=smote.fit_sample(X_train,y_train)
model_compare(rfc,X_train_sm,X_test,y_train_sm,y_test)

# checking whether sample is as same as population
-a.	Are both train and test representative of the overall data? How would you ascertain this statistically

In [ ]:
#one sample t test:

#we shall use one sample t test to check the hypothesis:
#fi nd mean and std deviation of the target variable from dataframe
print('Mean is %2.1f Sd is %2.1f' % (df['Churn'].mean(),np.std(df['Churn'],ddof = 1)))

from scipy.stats             import ttest_1samp,ttest_ind, wilcoxon
from statsmodels.stats.power import ttest_power


#first check y_train with above mean
t_statistic, p_value = ttest_1samp(y_train, df['churn'].mean())
print(t_statistic, p_value)

#second, check y_test with above mean
t_statistic, p_value = ttest_1samp(y_test, df['churn'].mean())
print(t_statistic, p_value)

#explain about hypothesis. 
#ho: means of sample and population are same.
#h1: means are different. we need h0 to be accpeted here.

# Skeleton for classification  probelm:

In [ ]:
from pandas import DataFrame
def function(a,b,c,d,A,B,C,D):
    Index=['RMSE','R^2','Adj R^2','Data size']
    data={'Train':[a,b,c,d], 'Test':[A,B,C,D]}
    df3=DataFrame(data,columns=['Train','Test'],index=Index)
    print(df3)

In [ ]:
#for train
a=rmse1
b=r21
c=model_1.rsquared_adj
d=X_train.size

#for test
A=np.sqrt(MSE)
B=r2
C=model_11.rsquared_adj
D=X_test.size


function(rmse1,r21,model_1.rsquared_adj,X_train.size,np.sqrt(MSE),r2,model_11.rsquared_adj,X_test.size)


# Error

In [ ]:
when we get 
linalgerror singular matrix

do
result=model.fit(method='bfgs')

# to silent the warnings

In [ ]:
import warnings
warnings.simplefilter('ignore')